In [4]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from yolo_weighted_dataloader import YOLOWeightedDataset
from ultralytics import YOLO

def verify_class_balance(dataset, num_samples=1000):
    """
    Verifies whether the __getitem__ method in the YOLOWeightedDataset class returns a balanced class output.

    Args:
        dataset: An instance of YOLOWeightedDataset.
        num_samples: Number of samples to draw from the dataset.

    Returns:
        class_counts: A dictionary containing the class counts.
    """
    all_labels = []
    num_samples = min(len(dataset.labels), num_samples)

    if dataset.train_mode:
        choices = np.random.choice(len(dataset.labels), size=num_samples, p=dataset.probabilities)
    else:
        choices = np.random.choice(len(dataset.labels), size=num_samples, replace=False)

    for i in choices:
        label = dataset.labels[i]["cls"]
        all_labels.extend(label.reshape(-1).astype(int))

    class_counts = Counter(all_labels)
    return class_counts

def plot_class_balance(weighted_cnts, unweighted_cnts, class_names):
    """
    Plots the comparison of class distribution between training and validation modes.

    Args:
        weighted_cnts: A dictionary containing the class counts in weighted mode.
        unweighted_cnts: A dictionary containing the class counts in unweighted mode.
        class_names: A list of class names.
    """
    classes = range(len(class_names))
    weighted_values = [weighted_cnts.get(c, 0) for c in classes]
    unweighted_values = [unweighted_cnts.get(c, 0) for c in classes]

    width = 0.35  # Bar width

    fig, ax = plt.subplots()
    ax.bar(classes, unweighted_values, width, label='Normal mode')
    ax.bar([c + width for c in classes], weighted_values, width, label='Weighted Mode')

    ax.set_xlabel('Class')
    ax.set_ylabel('Count')
    ax.set_title('Class Distribution in Normal vs Weighted Modes')
    ax.set_xticks([c + width / 2 for c in classes])
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.legend()

    plt.show()

In [6]:
model = YOLO("/home/nele_pauline_suffo/models/yolov11_face_detection.pt")
build.YOLODataset = YOLOWeightedDataset
# Get class counts in weighted mode
model.trainer.train_loader.dataset.train_mode = True
weighted_counts = verify_class_balance(model.trainer.train_loader.dataset, num_samples=1000)

AttributeError: 'NoneType' object has no attribute 'train_loader'

In [2]:
import os
images_with_class_count = len([f for f in os.listdir("/home/nele_pauline_suffo/ProcessedData/yolo_face_labels") if f.endswith('.txt')])
images_with_class_count

17136

In [1]:
import os
images_with_class_count = len([f for f in os.listdir("/home/nele_pauline_suffo/ProcessedData/yolo_person_labels") if f.endswith('.txt')])
images_with_class_count

88728

In [6]:
import os
images_with_class_count = len(os.listdir("/home/nele_pauline_suffo/ProcessedData/quantex_videos_processed/quantex_at_home_id262565_2022_05_26_02"))
images_with_class_count

1799